In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

In [ ]:
from AutoScreen.AndroidScreen import ADBPropertiesHelper, AndroidController

In [ ]:
android_controller = AndroidController()
screenshot_raw = android_controller.get_screenshot("wechat_moment", "data/temp")
xml_raw = android_controller.get_xml("wechat_moment", "data/temp")

In [ ]:
from IPython.display import display, Image
display(Image('data/temp/wechat_moment.png'))

In [ ]:
class UINode:
    def __init__(self, uid, bbox, attrib):
        self.name = name
        self.bbox = bbox
        self.attrib = attrib

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse("data/temp/wechat_moment.xml")
root = tree.getroot()

In [ ]:
ui_node_list = []
for ele in tree.iter():
    if ele.attrib.get("clickable", "false") == "true":
    # if ele.attrib.get("clickable", "false") == "true" or ele.attrib.get("focusable", "false") == "true":
        print(ele, ele.attrib)
        bounds = ele.attrib["bounds"][1:-1].split("][")
        x1, y1 = map(int, bounds[0].split(","))
        x2, y2 = map(int, bounds[1].split(","))
        name = ele.attrib.get("resource-id", "no-resource-id").split('/')[-1]
        if name is None:
            name = ele.attrib.get("content-desc", "no-content-desc")
        ui_node_list.append(UINode(name, ((x1, y1), (x2, y2)), ele.attrib))

In [ ]:
len(ui_node_list)

In [ ]:
ui_node_list

In [ ]:
import os
import cv2
from matplotlib.colors import CSS4_COLORS, to_rgb
import random

In [ ]:
def add_text_with_transparent_bg(image, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, text_color=(255, 255, 255), bg_color=(128, 128, 128), alpha=0.5):
    # Calculate text size
    text_size = cv2.getTextSize(text, font, font_scale, 2)[0]
    # Calculate background rectangle coordinates
    x, y = position
    bg_rect_start = (x, y)
    bg_rect_end = (x + text_size[0] + 10, y + text_size[1] + 10)
    # Create overlay for transparency
    overlay = image.copy()
    cv2.rectangle(overlay, bg_rect_start, bg_rect_end, bg_color, -1)
    # Blend image and overlay with transparency
    image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)
    # Add text over rectangle
    cv2.putText(image, text, (x + 5, y + 30), font, font_scale, text_color, 2, bottomLeftOrigin=False)
    return image


In [ ]:
def draw_bbox_multi(img_path, output_path, elem_list):
    imgcv = cv2.imread(img_path)
    count = 1
    for elem in elem_list:
        try:
            top_left = elem.bbox[0]
            bottom_right = elem.bbox[1]
            left, top = top_left[0], top_left[1]
            right, bottom = bottom_right[0], bottom_right[1]
            label = str(count)
            label = str(elem.name)
            text_color = to_rgb(random.choice(list(CSS4_COLORS.values())))
            text_color = tuple(int(round(c * 255)) for c in text_color)
            rectangle_shrink = 5
            imgcv = add_text_with_transparent_bg(imgcv, label, (left + rectangle_shrink, top + rectangle_shrink), text_color=text_color)
            imgcv = cv2.rectangle(imgcv, (left + rectangle_shrink, top + rectangle_shrink), (right - rectangle_shrink, bottom - rectangle_shrink), text_color, 2)
        except Exception as e:
            logging.error(f"ERROR: An exception occurs while labeling the image\n{e}")
        count += 1
    cv2.imwrite(output_path, imgcv)
    return

In [ ]:
draw_bbox_multi(screenshot_raw, "data/temp/with_box.png", ui_node_list)

In [ ]:
from IPython.display import display, Image
display(Image('data/temp/with_box.png'))

In [ ]:
def get_box_from_bounds_raw_str(bounds: str):
    bounds = bounds.replace('[', '').replace(']', ',').split(',')
    bounds = map(int, bounds[:4])
    left, high, right, low = bounds
    return left, high, right, low

In [ ]:
root.findall(".//*[@resource-id='com.tencent.mm:id/n9a']")

In [ ]:
tree.findall(".//*[@resource-id='com.tencent.mm:id/n9a']")

In [ ]:
moment_node_list = tree.findall(".//*[@resource-id='com.tencent.mm:id/n9a']")

In [ ]:
this_moment_node = moment_node_list[1]

In [ ]:
this_moment_tree = ET.ElementTree(this_moment_node)
this_moment_tree.write('data/temp/this_moment_tree.xml')

In [ ]:
this_moment_node

In [ ]:
# this_avatar_node = this_moment_node.find(".//*[@resource-id='com.tencent.mm:id/od']")
this_avatar_node = this_moment_node.find("*[@resource-id='com.tencent.mm:id/od']")


In [ ]:
# bounds = this_avatar_node.attrib.get("bounds", "bounds")
# bounds = bounds.replace('[', '').replace(']', ',').split(',')
# bounds = map(int, bounds[:4])
left, high, right, low = get_box_from_bounds_raw_str(bounds = this_avatar_node.attrib.get("bounds"))


In [ ]:
this_avatar_image = cv2.imread(screenshot_raw)[high:low, left:right]

In [ ]:
cv2.imwrite("data/temp/this_avatar_image.png", this_avatar_image)

In [ ]:
from IPython.display import display, Image
display(Image('data/temp/this_avatar_image.png'))